In [1]:
import configparser
import os
import pandas as pd
import math
import psycopg2
import numpy as np
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# DB setup
DB_CONFIG = {
    "dbname": "Data_analytics",
    "user": "postgres",
    "password": "1234",
    "host": "127.0.0.1",
    "port": "5432"
}

# SQL query 
query = """
WITH params AS (
    SELECT
        DATE '2025-06-15' AS redesign_date
),
fo AS (
    SELECT
        f.order_id,
        f.user_id,
        f.order_date::date AS order_date,
        CASE
            WHEN LOWER(dos.status_name) IN ('paid', 'completed', 'delivered', 'success', 'approved') THEN 'paid'
            WHEN LOWER(dos.status_name) IN ('refund', 'refunded', 'returned', 'chargeback') THEN 'refund'
            ELSE 'other'
        END AS status_group
    FROM fact.fact_orders f
    JOIN dim.dim_order_status dos ON dos.status_id = f.status_id
),
labeled AS (
    SELECT
        CASE
            WHEN fo.order_date < p.redesign_date THEN 'before'
            ELSE 'after'
        END AS period,
        du.device,
        du.region,
        du.channel,
        (CASE WHEN fo.status_group = 'paid' THEN 1 ELSE 0 END) AS is_order,
        (CASE WHEN fo.status_group = 'refund' THEN 1 ELSE 0 END) AS is_refund
    FROM fo
    JOIN dim.dim_user du ON du.user_id = fo.user_id
    CROSS JOIN params p
    WHERE fo.order_date BETWEEN (p.redesign_date - INTERVAL '30 day') AND (p.redesign_date + INTERVAL '30 day')
),
agg AS (
    SELECT
        device,
        region,
        channel,
        period,
        SUM(is_order) AS orders,
        SUM(is_refund) AS refunds
    FROM labeled
    GROUP BY device, region, channel, period
),
pivot AS (
    SELECT
        device,
        region,
        channel,
        SUM(orders)  FILTER (WHERE period = 'before') AS orders_before,
        SUM(refunds) FILTER (WHERE period = 'before') AS refunds_before,
        SUM(orders)  FILTER (WHERE period = 'after')  AS orders_after,
        SUM(refunds) FILTER (WHERE period = 'after')  AS refunds_after
    FROM agg
    GROUP BY device, region, channel
),
calc AS (
    SELECT
        device,
        region,
        channel,
        orders_before,
        refunds_before,
        orders_after,
        refunds_after,
        CASE WHEN orders_before > 0 THEN ROUND(100.0 * refunds_before::numeric / orders_before, 2) ELSE NULL END AS return_rate_percent_before,
        CASE WHEN orders_after > 0  THEN ROUND(100.0 * refunds_after::numeric / orders_after, 2)  ELSE NULL END AS return_rate_percent_after,
        CASE
            WHEN orders_before > 0 AND orders_after > 0
            THEN ROUND( (100.0 * refunds_after::numeric / orders_after) - (100.0 * refunds_before::numeric / orders_before), 2)
            ELSE NULL
        END AS delta_pp
    FROM pivot
)
SELECT
    device,
    region,
    CASE
        WHEN LOWER(channel) IN ('paid','ads','cpi','cpp','cpa','sem','facebook_ads','google_ads') THEN 'Paid'
        WHEN LOWER(channel) IN ('organic','seo','direct','referral') THEN 'Organic'
        ELSE 'Other'
    END AS channel_group,
    orders_before,
    refunds_before,
    orders_after,
    refunds_after,
    return_rate_percent_before,
    return_rate_percent_after,
    delta_pp
FROM calc
ORDER BY device, region, channel_group;
"""

# Load the retrieved dataset into a Pandas DataFrame for further analysis
conn = psycopg2.connect(**DB_CONFIG)
df = pd.read_sql(query, conn)
conn.close()

C:\Users\panty\AppData\Local\Temp/ipykernel_43012/4151672612.py:107: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [3]:
# Perform z-test for each segment

results = []

for _, row in df.iterrows():
    count = [row["refunds_before"], row["refunds_after"]]
    nobs = [row["orders_before"], row["orders_after"]]

    stat, pval = proportions_ztest(count, nobs)
    
    p1 = count[0] / nobs[0]
    p2 = count[1] / nobs[1]
    diff = p2 - p1
    se = np.sqrt(p1*(1-p1)/nobs[0] + p2*(1-p2)/nobs[1])
    ci_low, ci_high = diff - 1.96*se, diff + 1.96*se

    results.append({
        "device": row["device"],
        "region": row["region"],
        "channel_group": row["channel_group"],
        "return_rate_before": p1*100,
        "return_rate_after": p2*100,
        "delta_pp": diff*100,
        "z_stat": stat,
        "p_value": pval,
        "ci_low_pp": ci_low*100,
        "ci_high_pp": ci_high*100
    })

results_df = pd.DataFrame(results)

In [4]:
results_df

,device,region,channel_group,return_rate_before,return_rate_after,delta_pp,z_stat,p_value,ci_low_pp,ci_high_pp
0,Android,ASIA,Organic,6.833713,12.903226,6.069513,-2.576341,9.985221e-03,1.023050,11.115976
1,Android,ASIA,Paid,8.057851,16.129032,8.071181,-3.330972,8.654323e-04,2.890933,13.251429
2,Android,EU,Organic,8.341143,13.768116,5.426973,-3.421438,6.229102e-04,2.108056,8.745889
3,Android,EU,Paid,7.215620,51.558753,44.343133,-19.913696,3.096061e-88,39.323960,49.362307
4,Android,US,Organic,7.266436,13.804714,6.538278,-3.125417,1.775535e-03,2.080745,10.995810
5,Android,US,Paid,8.047690,12.023460,3.975770,-2.049762,4.038762e-02,-0.043347,7.994888
6,iOS,ASIA,Organic,7.373272,6.666667,-0.706605,0.326396,7.441248e-01,-4.881271,3.468061
7,iOS,ASIA,Paid,6.459330,8.333333,1.874003,-0.855812,3.921017e-01,-2.591199,6.339205
8,iOS,EU,Organic,6.016598,6.896552,0.879954,-0.453166,6.504292e-01,-3.010927,4.770836
9,iOS,EU,Paid,4.859813,7.364341,2.504528,-1.428210,1.532313e-01,-1.166704,6.175760


1) Android segments show statistically significant growth in return rate after the redesign, and especially Android + EU + Paid (a +44 pp spike, which is extreme);

2) iOS segments remain stable: no statistically significant shifts;

3) Confidence Intervals, which stay entirely above zero, show that the increase for Android users is statistically real rather than random noise;

→ this pattern shows that the problem is likely linked to the Android version of the new checkout UI, and not a global or backend problem